In [1]:
import logging
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.types import *

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
spark = (SparkSession.builder.appName("itoss-ai")
         .config("spark.jars","./postgresql-42.7.3.jar")
         .getOrCreate())

24/07/22 12:33:18 WARN Utils: Your hostname, fmalaspina-M4 resolves to a loopback address: 127.0.1.1; using 192.168.101.200 instead (on interface enp1s0)
24/07/22 12:33:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/07/22 12:33:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
jdbcUrl = "jdbc:postgresql://localhost:5432/itossconfig"
connection_properties = {
    "driver": "org.postgresql.Driver",
    "user": "itoss",
    "password": "admin"
}
query = "(select c.*, w.name as ctWorkgroup, l.name as ctLocation from ct c \
         inner join workgroup w on c.workgroup_id = w.id \
         inner join location l on c.location_id = l.id \
         inner join ct_type ct on c.type_id = ct.id \
) as subquery"  
ctDf = spark.read.jdbc(url=jdbcUrl,table=query,properties=connection_properties)
ctDf.printSchema()




root
 |-- id: long (nullable = true)
 |-- created_by: string (nullable = true)
 |-- creation_date: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_modified_date: timestamp (nullable = true)
 |-- attributes: string (nullable = true)
 |-- key: string (nullable = true)
 |-- environment: string (nullable = true)
 |-- name: string (nullable = true)
 |-- old_password: string (nullable = true)
 |-- state: string (nullable = true)
 |-- collector_id: long (nullable = true)
 |-- company_id: long (nullable = true)
 |-- contact_id: long (nullable = true)
 |-- location_id: long (nullable = true)
 |-- monitoring_profile_id: long (nullable = true)
 |-- support_user_id: long (nullable = true)
 |-- type_id: long (nullable = true)
 |-- workgroup_id: long (nullable = true)
 |-- instrumentation_parameter_values: string (nullable = true)
 |-- old_crypted_property_values: string (nullable = true)
 |-- integration_id: string (nullable = true)
 |-- ctworkgroup: string (nu

In [8]:
query = "(select ct.*, rev.revtstmp from itoss_audit.ct_aud ct inner join itoss_audit.revinfo rev \
 on ct.rev = rev.rev) as subquery"  
df = spark.read.jdbc(url=jdbcUrl,table=query,properties=connection_properties)
df.show(5, truncate=False)



+------+-------+-------+---------------+-----------------------+----------------+-----------------------+----------------------------------------------------+-----------------------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------------------+----------------------------------------+------------+----------+------------+----------+----------+-----------+---------------------+---------------+-------+------------+-------------+
|id    |rev    |revtype|created_by     |creation_date          |last_modified_by|last_modified_date     |attributes                                          |key                                |environment|instrumentation_parameter_val